### Assignment_8.11442

Read the following data set: 
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/ 

Rename the columns as per the description from this file: 
https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names 

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as db

from pandasql import sqldf

pysqldf = lambda q: sqldf(q, globals())

In [2]:
adult_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", 
                       names=['age','workclass','fnlwgt','education','education-num',
                              'marital-status','occupation','relationship','race','sex',
                              'capital-gain','capital-loss','hours-per-week','native-country','income_range'])

In [3]:
adult_df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_range
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


#### Create a sql db from adult dataset and name it sqladb 

In [4]:
conn = db.connect('sqldb.db')
cur = conn.cursor()

sql = """
    create table if not exists adult('age','workclass','fnlwgt','education','education_num',
    'marital_status','occupation','relationship','race','sex','capital_gain',
    'capital_loss','hours_per_week','native_country','range')
    """
ins_query = "insert into adult values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
conn.execute(sql)


conn.executemany(ins_query, adult_df.values.tolist())


#### 1. Select 10 records from the adult sqladb

In [5]:
row = conn.execute("select * from adult limit 10;")
for r in row:
    print(r)

(39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K')
(50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K')
(38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K')
(28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K')
(37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
(49, ' Private', 160187, ' 9th', 5, ' Married-spouse-absent', ' Other-service', ' N

#### 2. Show me the average hours per week of all men who are working in private sector 

In [6]:
row = conn.execute("Select avg(hours_per_week) from adult where trim(workclass) = 'Private' and trim(sex) = 'Male';")
for r in row:
    print("Average hours per week of all men who are working in private sector is: ", r[0])

Average hours per week of all men who are working in private sector is:  42.22122591006424


#### 3. Show me the frequency table for education, occupation and relationship, separately 

In [7]:
sql = "select education, count (education) from adult group by education"
data = conn.execute(sql)

ed = []
fr = []
for row in data:
    ed.append(row[0])
    fr.append(row[1])

pd.DataFrame({"Education":ed, "Frequency":fr})

,Education,Frequency
0,10th,933
1,11th,1175
2,12th,433
3,1st-4th,168
4,5th-6th,333
5,7th-8th,646
6,9th,514
7,Assoc-acdm,1067
8,Assoc-voc,1382
9,Bachelors,5355


In [8]:
sql = "select occupation, count (occupation) from adult group by occupation"
data = conn.execute(sql)

ed = []
fr = []
for row in data:
    ed.append(row[0])
    fr.append(row[1])

pd.DataFrame({"Occupation":ed, "Frequency":fr})

,Occupation,Frequency
0,?,1843
1,Adm-clerical,3770
2,Armed-Forces,9
3,Craft-repair,4099
4,Exec-managerial,4066
5,Farming-fishing,994
6,Handlers-cleaners,1370
7,Machine-op-inspct,2002
8,Other-service,3295
9,Priv-house-serv,149


In [9]:
sql = "select relationship, count (relationship) from adult group by relationship"
data = conn.execute(sql)

ed = []
fr = []
for row in data:
    ed.append(row[0])
    fr.append(row[1])

pd.DataFrame({"Relationship":ed, "Frequency":fr})

,Relationship,Frequency
0,Husband,13193
1,Not-in-family,8305
2,Other-relative,981
3,Own-child,5068
4,Unmarried,3446
5,Wife,1568


#### 4. Are there any people who are married, working in private sector and having a masters                degree 

In [10]:
sql = """select count(*) from adult 
        where trim(workclass) = 'Private' and trim(education) = 'Masters' 
        and trim(marital_status) like 'Married-%'"""
data = conn.execute(sql)
for d in data:
    print("People who are married, working in private sector and having a masters degree:", d[0])

People who are married, working in private sector and having a masters degree: 540


In [11]:
sql = """select * from adult 
        where trim(workclass) = 'Private' and trim(education) = 'Masters' 
        and trim(marital_status) like 'Married-%'"""
data = conn.execute(sql)
for d in data:
    print(d)

(37, ' Private', 284582, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Wife', ' White', ' Female', 0, 0, 40, ' United-States', ' <=50K')
(33, ' Private', 202051, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' White', ' Male', 0, 0, 50, ' United-States', ' <=50K')
(76, ' Private', 124191, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 40, ' United-States', ' >50K')
(31, ' Private', 99928, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' White', ' Female', 0, 0, 50, ' United-States', ' <=50K')
(35, ' Private', 138992, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Other-relative', ' White', ' Male', 7298, 0, 40, ' United-States', ' >50K')
(34, ' Private', 142897, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' Asian-Pac-Islander', ' Male', 7298, 0, 35, ' Taiwan', ' >50K')
(62, ' Private', 270092, ' Masters', 14, ' Married-civ-spouse', ' Prof-s

(31, ' Private', 263561, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 40, ' United-States', ' >50K')
(54, ' Private', 68985, ' Masters', 14, ' Married-civ-spouse', ' Sales', ' Husband', ' White', ' Male', 0, 0, 55, ' United-States', ' >50K')
(47, ' Private', 120781, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' Asian-Pac-Islander', ' Male', 15024, 0, 40, ' ?', ' >50K')
(33, ' Private', 55291, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 40, ' United-States', ' >50K')
(57, ' Private', 112840, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband', ' White', ' Male', 0, 0, 55, ' United-States', ' >50K')
(51, ' Private', 162238, ' Masters', 14, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 1887, 47, ' United-States', ' >50K')
(68, ' Private', 208478, ' Masters', 14, ' Married-civ-spouse', ' Prof-specialty', ' Husband'

#### 5. What is the average, minimum and maximum age group for people working in              different sectors 

In [12]:
sql = "select workclass, avg(age), min(age), max(age) from adult group by workclass" 
data = conn.execute(sql)

wc = []
avg_age = []
min_age = []
max_age = []
for row in data:
    wc.append(row[0])
    avg_age.append(row[1])
    min_age.append(row[2])
    max_age.append(row[3])

pd.DataFrame({"Workclass":wc, "Average_Age":avg_age, "Min_Age":min_age, "Max_Age":max_age})

,Workclass,Average_Age,Min_Age,Max_Age
0,?,40.960240,17,90
1,Federal-gov,42.590625,17,90
2,Local-gov,41.751075,17,90
3,Never-worked,20.571429,17,30
4,Private,36.797585,17,90
5,Self-emp-inc,46.017025,17,84
6,Self-emp-not-inc,44.969697,17,90
7,State-gov,39.436055,17,81
8,Without-pay,47.785714,19,72


#### 6. Calculate age distribution by country 

In [13]:
sql = "select native_country, age, count(*) from adult group by native_country, age" 
data = conn.execute(sql)

country = []
age = []
count = []
for row in data:
    country.append(row[0])
    age.append(row[1])
    count.append(row[2])

pd.DataFrame({"Country":country, "Age":age, "Age Grp Count":count})

,Country,Age,Age Grp Count
0,?,17,2
1,?,18,8
2,?,19,5
3,?,20,10
4,?,21,11
5,?,22,12
6,?,23,6
7,?,24,14
8,?,25,11
9,?,26,18


#### 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and             'capital-loss' 

In [14]:
sql = "select capital_gain, capital_loss, (capital_gain - capital_loss) as 'Net-Capital-Gain' from adult" 
data = conn.execute(sql)

cg = []
cl = []
ncg = []
for row in data:
    cg.append(row[0])
    cl.append(row[1])
    ncg.append(row[2])

pd.DataFrame({"capital_gain":cg, "capital_loss":cl, "Net-Capital-Gain":ncg})

,capital_gain,capital_loss,Net-Capital-Gain
0,2174,0,2174
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
5,0,0,0
6,0,0,0
7,0,0,0
8,14084,0,14084
9,5178,0,5178


In [15]:
conn.close()